# Okamoto-Tanaka Revisited protocol


In [1]:
import random
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import math

KEY_SIZE = 512
H1_EXPONENT_SIZE = 256
SESSION_KEY_SIZE = 128
EPHEMERAL_EXPONENT_SIZE = 256
ID_SIZE = 256
ENDIAN = "little"


In [2]:
#dobór funkcji haschującej
def select_SHA_context(length_in_bits):
    if length_in_bits <= 224:
        return hashes.SHA224()
    elif length_in_bits <= 256:
        return hashes.SHA256()
    elif length_in_bits <= 384:
        return hashes.SHA384()
    elif length_in_bits <= 512:
        return hashes.SHA512()
    else:
        #TODO Exception
        print("Error, output to big for now")

#hf_KGC = select_SHA_context(H1_EXPONENT_SIZE)
#hf_USER = select_SHA_context(SESSION_KEY_SIZE)


In [3]:
def eea(_a,_b):
    u1 = 1
    v1 = 0
    u2 = 0
    v2 = 1
    if _a < 1 or _b < 1:
        return "Error"
    
    if _b>_a:
        a = _b
        b = _a
    else:
        a = _a
        b = _b
    r = _b
    while r!=0:
        g = r
        q = a//b
        
        r = a - (q*b)   
        u = u1 - (q*u2)
        v = v1 - (q*v2)
        u1 = u2
        v1 = v2
        u2 = u
        v2 = v
        
        a = b
        b = r
        
        #print(q,r,u,v)
    return (a, u1, v1)

#print(eea(77, 11))
#print(eea(62, 7))

## Generacja klucza

### Generacja klucza RSA dla KGC:

In [4]:
class KGC_key_public:
    def __init__(self, N, e, QRN_generator, h1sf, h2f, key_size):
        self.N = N
        self.e = e
        self.QRN_generator = QRN_generator
        self.h1sf = h1sf
        self.h2f = h2f
        self.key_size = key_size

class KGC_key:
    def __init__(self, h1sf, h2f,key_size = KEY_SIZE):
        
        rsa_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size = key_size,
            backend = default_backend()
        )
        self.key_size = key_size
        self.N = rsa_key.public_key().public_numbers().n
        self.e = rsa_key.public_key().public_numbers().e
        self.d = rsa_key.private_numbers().d
        p = rsa_key.private_numbers().p
        q = rsa_key.private_numbers().q
        self.Phi = (p-1)*(q-1)
        self.h1sf = h1sf # część funkcji hashującej H (wynik całej H musi być w QRN)
        self.h2f = h2f #druga funkcja hashująca jest w całości
    def set_QRN_generator(self, generator):
        self.QRN_generator = generator
    def public(self):
        #return (self.N,self.e,self.generator, self.h1sf, self.h2f)
        return KGC_key_public(self.N, self.e, self.QRN_generator, self.h1sf, self.h2f, self.key_size)
    # d,Phi - traktować jak prywatne

hash1_subfunction = select_SHA_context(H1_EXPONENT_SIZE)
hash2_function = select_SHA_context(SESSION_KEY_SIZE)
    
kgc01 = KGC_key(hash1_subfunction, hash2_function)
print('0x{:0128x}'.format(kgc01.N))
print('0x{:0128x}'.format(kgc01.e))
print('0x{:0128x}'.format(kgc01.d))
print(hex(kgc01.N))
        

0xa6a91af51a7b2100995a44ae3c9b31aabfb676dd23c8c67f4432d4f1dca9ee8011e5cfcb28ba255ba74b4476a04bbeefd29b4682d5c386e8001ec5150a0e4121
0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010001
0x57aa3214e59608f910b16bd91ee61f8d0a850727319085e121d57f2fadfbf108b11799aeeb4119ee261811ee7599596ed38b2b89498eeb1a7fae871bfdd95225
0xa6a91af51a7b2100995a44ae3c9b31aabfb676dd23c8c67f4432d4f1dca9ee8011e5cfcb28ba255ba74b4476a04bbeefd29b4682d5c386e8001ec5150a0e4121


In [5]:
def hashH1(material, kgc_public, endian = ENDIAN, id_size = ID_SIZE):
    material = material.to_bytes(id_size, endian)
    context = hashes.Hash(kgc_public.h1sf, backend=default_backend()) 
    context.update(material)
    exponent = int.from_bytes(context.finalize(), byteorder = endian)
    hsh = pow(kgc_public.QRN_generator, exponent, kgc_public.N)
    return hsh


### Generacja kluczy uczestników mOT
#### Znalezienie generatora g grupy reszt kwadratowych mod N
Chcemy znaleźć generator podgrupy reszt kwadratowych w grupue Z_N. Oczekujemy, że grupa reszt będzie odpowiednio duża. Podgrupę będziemy oznaczać __QRN__ (analogicznie do oznaczenia w pracy Gannaro-Krawczyka-Rabina), generator będzie oznaczany __g__.
___
##### Rozkład Phi = d * (2^r) 
Najpierw wyznaczamy rozkład wartości funkcji Eulera dla _modułu klucza RSA_ __KGC__. Chcemy znaleźć takie __t__ oraz __r__, że __Phi = t * (2^r)__, przy czym __t__ jest nieparzyste.

In [6]:
#Rozłożenie N-1 = d* (2**r)
def mr(number):
    t = number
    r = 0
    while t%2 == 0:
    #while math.fmod(d,2) ==0:
        t//=2
        r+=1
    return (t,r)

(t,r) = mr(kgc01.Phi)
print(t,r)
print(t*(2**r))


2182180546594429814613501326587981365002226486266829943459173121815678406817674541252002127076638819638819964579756794184281194254257401183697611575896801 2
8728722186377719258454005306351925460008905945067319773836692487262713627270698165008008508306555278555279858319027176737124777017029604734790446303587204


##### Wyznaczenie generatora g grupy QR_N
W celu wygenerowania akceptowalnego g losujemy pewnwe __alpha__ i sprawdzamy czy generuje podgrupę o satysfakcjonująco dużym rzędzie. Załóżmy tutaj, że zadowala nas jeżeli __alpha__ podniesione do potęgi t modulo N jest różne od 1.
W dalszej części podnosilibyśmy do kwadratu maksymalnie r razy. Jeżeli __alpha__ nie spełnia oczekiwań losujemy je ponownie i powtarzamy procedurę.  
_W przyszłości rozważyć szukanie większych podgrup (podnieść kilka razy do kwadratu i sprawdzić po ilu podniesieniach jest 1)._

In [7]:
#generator grupy Z_N: Znaleźć alpha że g = alpha**2 jest generatorem dużej podgrupy (reszt kwadratowych)
cont = True
while cont == True:
    alpha = random.randint(1,kgc01.N)
    #print (pow(alpha, Phi, N))
    beta = pow(alpha,t,kgc01.N)
    print (beta, alpha)
    if  beta != 1:
        cont = False
        #print ("alpha jest generatorem dużej podgrupy")
    
g = pow(alpha,2,kgc01.N)
kgc01.set_QRN_generator(g)
print (pow(kgc01.QRN_generator, kgc01.Phi, kgc01.N))

784341722290451240020670949797711354506350558383969377880287816098998994003483360748357131412938433623164322952887907809080449887070817361274441036206374 5141384625985612321398525402265837814420587247403326872509659045388233495794075294182241022714117255379064440805047160114306302385265366405806353762615253
1


#### Klasa użytkowników

In [8]:
#Użytkownik
class User:
    def __init__(self,name,user_id):
        self.name = name
        self.id = user_id #user_id.to_bytes(ID_SIZE//8, byteorder=ENDIAN)
        #self.idn = int.from_bytes(self.id, byteorder=ENDIAN)
        
    def set_private_key(self,key, endian = ENDIAN):
        self.private_key = key
        #self.private_key_n = int.from_bytes(key, endian)
    def generate_ephemeral_exponent(self, ephemeral_exponent_size = EPHEMERAL_EXPONENT_SIZE):
        self.ephemeral_exponent = random.randint(2**(ephemeral_exponent_size//4),2**(ephemeral_exponent_size))
    def calculate_shared_secred(self, msg, id_corr, kgc_public, endian = ENDIAN):
        #print("id_corr", id_corr)
        #hsh = hashH1(corr_idn.to_bytes(kgc_public.key_size//8, endian), kgc_public)
        hsh = hashH1(id_corr, kgc_public)
        rev_hsh = eea(kgc_public.N, hsh)[2]
        
        print("with reversing hsh:", hsh*rev_hsh%kgc_public.N)
        
        
        
        #msgn = int.from_bytes(msg,endian)
        #msgn_e = pow(msgn, kgc_public.e, kgc_public.N)
        msgn_e = pow(msg, kgc_public.e, kgc_public.N)
        
        t = (msgn_e * rev_hsh)%kgc_public.N
        K = pow(t, 2*self.ephemeral_exponent, kgc_public.N)
        
        return K
        
        
    
user_A = User("A",random.randint(1,2**ID_SIZE))
user_B = User("B",random.randint(1,2**ID_SIZE))

#### Generacja kluczy prywatnych uczestników mOT.

In [9]:
# funkcja hashująca do grupy reszt kwadratowych
def generate_user_ltk(kgc, user_id, endian = ENDIAN):
    #context = hashes.Hash(kgc.h1sf, backend=default_backend()) 
    #context.update(material)
    #exponent = int.from_bytes(context.finalize(), byteorder = endian)
    #sk = pow(kgc.QRN_generator, exponent, kgc01.N)
    hsh = hashH1(user_id, kgc.public(), endian)
    return pow(hsh, kgc.d, kgc.N)

user_A.set_private_key(generate_user_ltk(kgc01, user_A.id))
user_B.set_private_key(generate_user_ltk(kgc01, user_B.id))
print(user_A.private_key)
    

    

1719407684113511332635213324017993778130319746033775904373235795911743881178631711044664134853407642432529313215109498231278025887966014834765204907007194


##### Weryfikacja klucza 

In [10]:
usk = user_A.private_key
usk_pow_e = pow(usk,kgc01.e,kgc01.N)
#usk_pow_e = usk
hash_context = hashes.Hash(select_SHA_context(H1_EXPONENT_SIZE), backend=default_backend()) 
hash_context.update(user_A.id.to_bytes(ID_SIZE, byteorder = ENDIAN))
exponent = int.from_bytes(hash_context.finalize(), byteorder = ENDIAN)
sk_pre_d = pow(kgc01.QRN_generator, exponent, kgc01.public().N)
print ( sk_pre_d == usk_pow_e)
if(sk_pre_d != usk_pow_e):
    print (usk_pow_e,"\n", sk_pre_d )
    




True


## Przebieg sesji

### Generacja wiadomości


In [11]:

def generate_negotiation_message(user, kgc_public, endian = ENDIAN):
    g_p = pow(kgc_public.QRN_generator, user.ephemeral_exponent, kgc_public.N)
    return ((g_p*user.private_key)%kgc_public.N)
    

user_A.generate_ephemeral_exponent()
user_B.generate_ephemeral_exponent()

msg_A = generate_negotiation_message(user_A,kgc01.public())
msg_B = generate_negotiation_message(user_B,kgc01.public())

#print(msg_A)

    

#idA = bytes(user_A.id.to_bytes(ID_SIZE, byteorder="little"))
#idA = bytes(user_A.id.to_bytes(ID_SIZE, byteorder="big"))
print(user_A.id)



#print(idA)
#print(hex(user_A.id))

23627768970043766209963632374526829787678645994708545613609110006745414285329


In [12]:
k_A = user_A.calculate_shared_secred(msg_B, user_B.id, kgc01.public())
k_B = user_B.calculate_shared_secred(msg_A, user_A.id, kgc01.public())

k = pow(kgc01.QRN_generator,2*user_A.ephemeral_exponent*user_B.ephemeral_exponent*kgc01.e, kgc01.N)
#print (kgc01.QRN_generator)
print("k=\t", k)
print("k_A=\t", k_A)
print("k_B=\t", k_B)

k1a = pow(msg_B, kgc01.e, kgc01.N)
k2a = eea(kgc01.N,hashH1(user_B.id, kgc01.public()))[2]
k3a = k1a*k2a%kgc01.N
k4a = pow(k3a, 2*user_A.ephemeral_exponent, kgc01.N)
print("k1a =\t", k1a)
print("k2a =\t", k2a)
print("k3a =\t", k3a)
print("k4a =\t", k4a)



with reversing hsh: 1
with reversing hsh: 1
k=	 2850274805489043140087675445862511545602211962877090195245556840143571801922215910873561281336283663057425379228646242791665052687520723485361798336960526
k_A=	 2850274805489043140087675445862511545602211962877090195245556840143571801922215910873561281336283663057425379228646242791665052687520723485361798336960526
k_B=	 2850274805489043140087675445862511545602211962877090195245556840143571801922215910873561281336283663057425379228646242791665052687520723485361798336960526
k1a =	 8378656219709587316290902258174933378010436836431886404958396195085168125067806584616991366312981584521284089318120357632473914231771314191798250378533944
k2a =	 990471411591126867090953731246250150602150526367473641544397965010244907142931203174950398423509565314933786442838838871953980738461289610635389573321984
k3a =	 4161110457891112814497941313010733229682465451587736350716286579682170012263665735540712818629780736348919101785157638170805521998303023284137536